# Homework: Vector Search

## Q1. Embedding the query

Embed the query: **I just discovered the course. Can I join now?**. Use the `jinaai/jina-embeddings-v2-small-en` model.

You should get a numpy array of size 512.

In [1]:
from fastembed import TextEmbedding

import numpy as np

In [2]:
TextEmbedding.list_supported_models()

[{'model': 'BAAI/bge-base-en',
  'sources': {'hf': 'Qdrant/fast-bge-base-en',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.42,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model': 'BAAI/bge-base-en-v1.5',
  'sources': {'hf': 'qdrant/bge-base-en-v1.5-onnx-q',
   'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz',
   '_deprecated_tar_struct': True},
  'model_file': 'model_optimized.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.',
  'license': 'mit',
  'size_in_GB': 0.21,
  'additional_files': [],
  'dim': 768,
  'tasks': {}},
 {'model':

In [22]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
embedding_model = TextEmbedding(model_handle)

query = "I just discovered the course. Can I join now?"
embeddings = list(embedding_model.embed(query))

In [23]:
min_val = np.min(embeddings)
min_val

np.float64(-0.11726373885183883)

### What's the minimal value in this array?

The minimal value in this array is **-0.11**

### Cosine similarity

The vectors that our embedding model returns are already normalized: their length is 1.0.

You can check that by using the norm function:

In [24]:
np.linalg.norm(embeddings)

np.float64(1.0)

Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

In [26]:
embeddings[0].dot(embeddings[0])

np.float64(1.0000000000000002)

## Q2. Cosine similarity with another vector

Now let's embed this document:

In [27]:
doc = 'Can I still join the course after the start date?'

In [28]:
embeddings2 =  list(embedding_model.embed(doc))

In [29]:

np.linalg.norm(embeddings2)

np.float64(1.0)

In [30]:
embeddings[0].dot(embeddings2[0])

np.float64(0.9008528895674548)

### What's the cosine similarity between the vector for the query and the vector for the document?

The cosine similarity between the vector for the query 'I just discovered the course. Can I join now?' and the vecto for the document 'Can I still join the course after the start date?' is **0.9**

## Q3. Ranking by cosine

For Q3 and Q4 we will use these documents:

In [31]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

In [ ]:
for document in documents:
    text = document['text']
    embedding = list(embedding_model.embed(text))
    document["embedding"] = embedding
    document['cosine'] = embedding[0].dot(embeddings2[0])
    

In [35]:
cosine =  [d['cosine'] for d in documents]
cosine

[np.float64(0.8106057216700708),
 np.float64(0.8498906698471509),
 np.float64(0.7991335457966962),
 np.float64(0.6813047116438659),
 np.float64(0.6739288060941504)]

### What's the document index with the highest similarity? (Indexing starts from 0)

The document **2** with the index **1** with similarity **0.84** is have highest similarity

Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:

In [36]:
# V.dot(q)

If this hint is rather confusing you than helping, feel free to ignore it.

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of question and text:

In [39]:
doc = documents[0]
full_text = doc['question'] + ' ' + doc['text']
full_text

"Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

Embed this field and compute the cosine between it and the query vector.

In [42]:
for document in documents:
    full_text =  document['question'] + ' ' +  document['text']
    embedding = list(embedding_model.embed(full_text))
    document["embedding_full_text"] = embedding
    document['cosine_full_text'] = embedding[0].dot(embeddings2[0])

In [43]:
cosine_full_text =  [d['cosine_full_text'] for d in documents]
cosine_full_text

[np.float64(0.9273252252988442),
 np.float64(0.8594847511716719),
 np.float64(0.8396602144619353),
 np.float64(0.76320298378397),
 np.float64(0.7604449265387344)]

### A. What's the highest scoring document? 

The document **1** with the index **0** with similarity **0.95** is have highest similarity.

### B. Is it different from Q3?

Yes, it is different from Question 3.

### C. If yes, why?

Because the question in the documents is equal question in the query vector.

## Q5. Selecting the embedding model

Now let's select a smaller embedding model.

In [45]:
list_model = TextEmbedding.list_supported_models()
list_model_order = sorted(list_model, key=lambda x: x["dim"])
list_model_order[0]

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

### What's the smallest dimensionality for models in fastembed?

The smallest dimensionality for models in fastemebed is **384**

One of these models is `BAAI/bge-small-en`. Let's use it.

## Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

In [46]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

In [47]:
model_handle_q5 = "BAAI/bge-small-en"
embedding_model_q5 = TextEmbedding(model_handle_q5)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

After the data is inserted, use the question from Q1 for querying the collection.

In [49]:
# text = doc['question'] + ' ' + doc['text']

query = "I just discovered the course. Can I join now?"
embeddings_query = list(embedding_model_q5 .embed(query))

for document in documents:
    full_text =  document['question'] + ' ' +  document['text']
    embedding = list(embedding_model_q5.embed(full_text))
    document["embedding"] = embedding
    document['cosine'] = embedding[0].dot(embeddings_query [0])

### What's the highest score in the results? (The score for the first returned record):

The highest score in the results is **0,87**.

In [52]:
list_documents_order = sorted(documents, key=lambda x: x["cosine"], reverse=True)
list_documents_order[0]['cosine']

np.float32(0.8703172)